In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import time
import datetime

spark = SparkSession.builder.appName("MySparkSession").getOrCreate()


df = spark.read.parquet('../time_series/TS6/TS6-2023.parquet')
df = df.toPandas()
df.head()

C:\Users\Ticy\anaconda3\envs\fhv\lib\site-packages\pyspark\sql\pandas\conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,Name,year,month,day,hour,PULocationID,weekday,is_holiday,count,lat,lon,TempTime,countN,__index_level_0__
0,lyft,2023,1,1,0,3,6,True,12,40.864294,-73.84651,2023-01-01 00:00:00,15.000000,26872560
1,lyft,2023,1,1,1,3,6,True,28,40.864294,-73.84651,2023-01-01 01:00:00,15.333333,26872561
2,lyft,2023,1,1,2,3,6,True,19,40.864294,-73.84651,2023-01-01 02:00:00,14.000000,26872562
3,lyft,2023,1,1,3,3,6,True,26,40.864294,-73.84651,2023-01-01 03:00:00,12.333333,26872563
4,lyft,2023,1,1,4,3,6,True,17,40.864294,-73.84651,2023-01-01 04:00:00,9.333333,26872564


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder



df = df.drop(['year', 'TempTime', 'countN', '__index_level_0__'], axis=1)

label = LabelEncoder()
df['Name'] = label.fit_transform(df['Name'])

df = df.astype({'is_holiday': 'int'})

X_valid = df.drop('count', axis=1).values
y_valid = df['count'].values




scaler = MinMaxScaler()
X_valid_scaled = scaler.fit_transform(X_valid)

In [3]:
import pickle

start_time = time.time()

with open('lgbm.pickle', 'rb') as f:
    model = pickle.load(f)
    pred = model.predict(X_valid_scaled)
    r2 = r2_score(y_valid, pred)
    mae = mean_absolute_error(y_valid, pred)
    print(f'R2: {r2}')
    print(f'MAE: {mae}')


end_time = time.time()
execution_time = end_time - start_time
print("執行時間:", execution_time, "秒")

R2: 0.8575427096251164
MAE: 10.564724760083212
執行時間: 12.474220752716064 秒
